In [1]:
%pip install h2o

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 177.0 MB 131.3 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 91.5 MB/s eta 0:00:01
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=0533030546660b1c148809ab4577dcff647b0b881df5f2d4b3bf02c973cf12d2
  Stored in directory: /home/mk7516/.cache/pip/wheels/bf/d7/e0/905f059465fafaf07c9e79ba4c8d642ac646b01ea8c38b07d5
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=4f831c65e1c94dc6bcb6bd66006a00f5a1ba67ae0b30925cbe353bfd64aac98a
  Stored in directory: /home/mk7516/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built h2o future
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-18891165/tmpdlqaa_2p
  JVM stdout: /state/partition1/job-18891165/tmpdlqaa_2p/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-18891165/tmpdlqaa_2p/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_mk7516_6z0xms
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [4]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [5]:
test, train = dataSetup('airlines')

/home/mk7516/.local/lib/python3.8/site-packages/sklearn/datasets/_openml.py:417: UserWarning: Multiple active versions of the dataset matching the name airlines exist. Versions may be fundamentally different, returning version 1.
  warn(


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
train.shape

(404537, 8)

In [7]:
x = train.columns
y = "class"
x.remove(y)

In [8]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [9]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_3_AutoML_1_20220501_153607

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.1847787640101926
RMSE: 0.42985900480296163
LogLoss: 0.5488763137460702
Null degrees of freedom: 9936
Residual degrees of freedom: 9919
Null deviance: 13673.963424813815
Residual deviance: 10908.3678593894
AIC: 10944.3678593894
AUC: 0.7997663560964748
AUCPR: 0.8247391427885559
Gini: 0.5995327121929497

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4772564283512219: 


,,0,1,Error,Rate
0,0,2395.0,2070.0,0.4636,(2070.0/4465.0)
1,1,725.0,4747.0,0.1325,(725.0/5472.0)
2,Total,3120.0,6817.0,0.2813,(2795.0/9937.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.477256,0.772561,234.0
1,max f2,0.230732,0.873319,341.0
2,max f0point5,0.574722,0.759089,178.0
3,max accuracy,0.537781,0.729898,199.0
4,max precision,0.959875,1.000000,0.0
5,max recall,0.103830,1.000000,388.0
6,max specificity,0.959875,1.000000,0.0
7,max absolute_mcc,0.565873,0.454590,183.0
8,max min_per_class_accuracy,0.565873,0.727436,183.0
9,max mean_per_class_accuracy,0.565873,0.728210,183.0



Gains/Lift Table: Avg response rate: 55.07 %, avg score: 55.51 %


...


Gains/Lift Table: Avg response rate: 55.45 %, avg score: 55.45 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010002,0.896733,1.659557,1.659557,0.920168,0.913436,0.920168,0.913436,0.016598,0.016598,65.955734,65.955734,0.014806
1,2,0.020001,0.876846,1.600221,1.629893,0.887268,0.886112,0.903720,0.899775,0.016001,0.032599,60.022126,62.989296,0.028277
2,3,0.030002,0.861619,1.558816,1.606199,0.864310,0.868802,0.890583,0.889450,0.015591,0.048189,55.881601,60.619869,0.040821
3,4,0.040001,0.849365,1.529328,1.586983,0.847960,0.855462,0.879928,0.880954,0.015292,0.063481,52.932820,58.698345,0.052701
4,5,0.050000,0.838343,1.511493,1.571887,0.838072,0.843782,0.871558,0.873520,0.015114,0.078595,51.149347,57.188694,0.064180
5,6,0.100001,0.796370,1.463819,1.517853,0.811638,0.816251,0.841598,0.844886,0.073192,0.151786,46.381916,51.785305,0.116233
6,7,0.150001,0.761342,1.398016,1.477907,0.775152,0.778404,0.819449,0.822725,0.069901,0.221688,39.801551,47.790720,0.160900
7,8,0.200001,0.730510,1.331677,1.441350,0.738370,0.745601,0.799179,0.803444,0.066584,0.288272,33.167687,44.134962,0.198123
8,9,0.300000,0.676406,1.262070,1.381591,0.699775,0.702712,0.766045,0.769867,0.126205,0.414477,26.207033,38.159084,0.256944
9,10,0.400000,0.627133,1.190217,1.333747,0.659935,0.651396,0.739517,0.740249,0.119023,0.533499,19.021685,33.374705,0.299638




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.653318,0.001802,0.653063,0.652119,0.653981,0.656031,0.651397
1,auc,0.725275,0.002471,0.725800,0.720918,0.726584,0.726905,0.726169
2,err,0.346682,0.001802,0.346937,0.347880,0.346019,0.343969,0.348603
3,err_count,28049.000000,124.893950,28096.000000,28091.000000,27949.000000,27899.000000,28210.000000
4,f0point5,0.670473,0.001828,0.670696,0.669324,0.670752,0.673216,0.668375
5,f1,0.746323,0.001264,0.747170,0.744234,0.746207,0.747436,0.746568
6,f2,0.841534,0.002910,0.843328,0.838025,0.840789,0.840048,0.845481
7,lift_top_group,1.660948,0.006705,1.659504,1.657710,1.657954,1.672812,1.656758
8,logloss,0.603234,0.002063,0.602313,0.606900,0.602418,0.601946,0.602591
9,max_per_class_error,0.678272,0.008968,0.683927,0.673254,0.674308,0.668906,0.690966



See the whole table with table.as_data_frame()


In [10]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_3_AutoML_1_20220501_153607,0.725267,0.603232,0.748256,0.377931,0.456297,0.208207
StackedEnsemble_AllModels_4_AutoML_1_20220501_153607,0.725254,0.603243,0.748221,0.380308,0.456301,0.208211
StackedEnsemble_AllModels_2_AutoML_1_20220501_153607,0.723156,0.604877,0.746204,0.37879,0.457071,0.208914
StackedEnsemble_AllModels_1_AutoML_1_20220501_153607,0.72306,0.60494,0.746065,0.380696,0.457099,0.208939
StackedEnsemble_BestOfFamily_4_AutoML_1_20220501_153607,0.722682,0.605148,0.745899,0.382598,0.457214,0.209045
StackedEnsemble_BestOfFamily_3_AutoML_1_20220501_153607,0.722306,0.605521,0.745275,0.384475,0.45737,0.209187
StackedEnsemble_BestOfFamily_2_AutoML_1_20220501_153607,0.722088,0.605661,0.745107,0.379756,0.457439,0.209251
StackedEnsemble_BestOfFamily_1_AutoML_1_20220501_153607,0.721769,0.60587,0.744841,0.382111,0.457539,0.209342
GBM_1_AutoML_1_20220501_153607,0.720667,0.606659,0.743892,0.383203,0.457918,0.209689
GBM_grid_1_AutoML_1_20220501_153607_model_9,0.719135,0.608059,0.742121,0.384779,0.458556,0.210274
